# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back', 'rb'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [90]:
import json
from os import listdir, path
import numpy as np
from sklearn import svm, metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import random

ROBOT_POSE_DATA_DIR = 'robot_pose_data_json'

In [91]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print (classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [92]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = json.load(open(filename, 'rb'))
    target = [i] * len(data)
    return data, target

In [69]:
# load all the data
all_data = []
all_target = []

for i in range(len(classes)):
    data, target = load_pose_data(i)
    all_data = all_data + data
    all_target = all_target + target
print ('total number of data', len(all_data))

total number of data 222


In [89]:
# shuffle data
permutation = np.random.permutation(len(all_data))

all_data = [all_data[i] for i in permutation]
all_target = [all_target[i] for i in permutation]

array([ 14, 138, 198, 179,  87, 158, 103, 208,  41,  20,  74, 173,  39,
       125,  95, 215,  46, 167, 204,  30,  66,  92,  12, 192, 172, 197,
       180, 106, 123, 143, 221,  48, 134, 206,  32,  73,  65, 136,  35,
       207, 141, 116, 168,  84, 186,  72,  71,  67, 177, 184, 135,  52,
        58,  47, 154, 183, 202, 159, 131, 127,  88,  36, 126, 156,  49,
       144,  29, 105,  86, 107,  42, 191, 212, 190, 115, 199, 121, 216,
       117,  78,  26, 196, 152,   3, 203, 182,  17,  75, 146, 100, 120,
       163, 220,  85,  13, 139,  79, 174,   1, 150,   7,  51, 132,  99,
        80,  59, 161,  82, 147, 195,  70, 149,  19,  68, 130, 124,  25,
       188,   9, 112, 218, 178,  90,  34, 189,  76,  96, 142, 153, 209,
       145,  38,   6,  55,  27,  89, 155, 162,  22,  56,  18,  64, 122,
       205, 113, 200, 219,  15,  28,  23, 109, 137,   4, 210,  57, 214,
       102,  83,  21, 114,  10,  62, 185, 176, 111,  98,  45,  43, 165,
       187, 217, 193,  91,  60,  40, 133, 129, 201,  31, 194, 16

In [26]:
n_training_data = int(len(all_data) * 0.7)
print('total number of training data', n_training_data)
X = all_data[:n_training_data]
y = all_target[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

### learning

In [27]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

### predicting

In [33]:
i = random.randint(n_training_data, len(all_data))
clf.predict([all_data[i]]), all_target[i]

(array([1]), 1)

In [34]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

## 4. Evaluate on the test data

In [35]:
expected = all_target[n_training_data:]
predicted = clf.predict(all_data[n_training_data:])
print(len(expected))
evaluate(expected, predicted)

67
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00         5
           4       1.00      1.00      1.00         4
           5       1.00      1.00      1.00         6
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         4
           8       1.00      1.00      1.00         7
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00        16

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67


Confusion matrix:
[[ 6  0  0  0  0  0  0  0  0  0  0]
 [ 0  4  0  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0  5  0  0  0  0  0  0  0

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [64]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [51]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb'))

In [65]:
i = random.randint(n_training_data, len(all_data))
clf2.predict(all_data[i]), all_target[i]

ValueError: Expected 2D array, got 1D array instead:
array=[ 0.30377388  0.01538205  0.42035794  1.98188615  0.30377388 -0.01683211
  0.51998401  1.71198606  0.         -0.08726692].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.